In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import  matplotlib.pyplot as plt
import pickle

In [25]:
def get_missing(df):
    missing = df.isna().sum()
    missing /= df.shape[0]
    missing *=100
    missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
    return missing


In [26]:
filepath = "datasets/Institutional Finances/F_F2_1415-1920_data.csv"
institutional_finance = pd.read_csv(filepath, index_col=0, low_memory= False)
missing = institutional_finance.isna().sum()
missing /= institutional_finance.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
threshold = 30
institutional_finance_over_threshold_missing = institutional_finance[(missing[missing["Percent Of Missing Values"] < threshold]).index]
missing = institutional_finance_over_threshold_missing.isna().sum()
missing /= institutional_finance_over_threshold_missing.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
# All missing entries with percent missing > 7 are verifiably 0 imputable
missing_keys = (missing[missing["Percent Of Missing Values"] > 7]).index
imputation_dict = {k: 0 for k in missing_keys}

cleaned_institutional_finance = institutional_finance_over_threshold_missing.fillna(imputation_dict)

# Remaining missing values are mean imputed -- very small fraction of rows in any case
num_cols = cleaned_institutional_finance.columns[cleaned_institutional_finance.dtypes.values != 'object']
cleaned_institutional_finance = institutional_finance_over_threshold_missing.fillna(value=cleaned_institutional_finance[num_cols].mean())
cleaned_institutional_finance = cleaned_institutional_finance[num_cols]
cleaned_institutional_finance.drop(columns= "f2d17", inplace = True)
sigma_threshold = 2
cleaned_institutional_finance_no_outliers = cleaned_institutional_finance[(np.abs(stats.zscore(cleaned_institutional_finance)) < sigma_threshold).all(axis=1)]

fnce_data_unique = cleaned_institutional_finance_no_outliers.groupby(['unitid']).mean().reset_index()
fnce_data_final = fnce_data_unique.drop(['year'], axis = 1)


In [27]:
def gen_expense():
    train_features = fnce_data_final.drop(['unitid'], axis = 1)
    model = PCA(n_components=train_features.shape[1]).fit(train_features)
    X_pc = model.transform(train_features)

    # number of components
    n_pcs= model.components_.shape[0]

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = train_features.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
    expenses = ['unitid']
    seen_categories = ['13']
    for i in range(n_pcs):
        most_important_name = most_important_names[i]
        category = most_important_name[3:5]

        if 'e' in most_important_name and most_important_name != 'unitid' and most_important_name not in expenses and category not in seen_categories:
            # print(f"name: {most_important_name}, cat: {category}")
            expenses.append(most_important_name)
            seen_categories.append(category)
    # expenses.append('unitid')
    expenses = np.array(expenses)



In [10]:
expenses = ['unitid', 'f2b02', 'f2c05', 'f2c06', 'f2d11', 'f2d12', 'f2e011', 'f2e021', 'f2e031', 'f2e041', 'f2e051', 'f2e061', 'f2e071', 'f2e081']

In [28]:
total_students = pd.read_csv("datasets/12-Month Enrollment/EFFY_2015-2021_data.csv", low_memory = False)
total_students = total_students.groupby('unitid').mean().reset_index()
total_students = total_students[['unitid', 'efytotlt']]

C:\Users\dragonblade62\AppData\Local\Temp\ipykernel_14188\927545629.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_students = total_students.groupby('unitid').mean().reset_index()


In [90]:
fnce_data_final_features = fnce_data_final[expenses]
for c in fnce_data_final_features.columns:
    if c != 'unitid':
        fnce_data_final_features[c] /= fnce_data_final['f2e131']
for c in fnce_data_final_features.columns:
    if c!= 'unitid' and 'e' in c:
        fnce_data_final_features = fnce_data_final_features[fnce_data_final_features[c] < 1]
fnce_data_final_features = pd.merge(fnce_data_final_features, total_students, on='unitid', how='inner')
fnce_data_final_features['expense_ps'] = fnce_data_final_features['f2b02'] / fnce_data_final_features['efytotlt']
fnce_data_final_features = fnce_data_final_features.drop(['efytotlt', 'f2b02'], axis = 1)

C:\Users\dragonblade62\AppData\Local\Temp\ipykernel_14188\1432335494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fnce_data_final_features[c] /= fnce_data_final['f2e131']


In [30]:
def test(X, y):
    # if np.count_nonzero(y == 0) > 0:
    #     lr = LinearRegression().fit(X, y)
    #     print(lr.score(X,y))
    # else:
    offset = int(X.shape[0] * 0.85)
    X_train, y_train = X[:offset], y[:offset]
    X_test, y_test = X[offset:], y[offset:]
    reg = LazyRegressor(verbose=1, ignore_warnings=True, custom_metric=None)
    models, predictions = reg.fit(X_train, X_test, y_train, y_test)
    print(models)


In [75]:
def xgb_test(X, y, input_frame, name, save_threshold = None, ):

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)
    grid = {
        'n_estimators': [3, 5, 7, 10, 15, 20, 30, 40],
        'max_depth' : [5, 7, 10, 12, 15, 18],
        'eta' : [0.05, 0.1, 0.12, 0.15, 0.17, 0.2, 0.22, 0.25],
        'grow_policy' : ['depthwise', 'lossguide']
    }

    model_s = XGBRegressor()
    xgbr_cv_s = RandomizedSearchCV(estimator=model_s, param_distributions=grid, cv=10, n_jobs=-1)
    xgbr_cv_s.fit(x_train, y_train)
    res_s = xgbr_cv_s.best_params_
    print(res_s)
    xgbr_s = XGBRegressor(n_estimators=res_s["n_estimators"], max_depth=res_s["max_depth"], eta=res_s["eta"], grow_policy=res_s["grow_policy"], eval_metric='rmse').fit(x_train, y_train)
    score = xgbr_s.score(x_test, y_test)
    preds = xgbr_s.predict(x_test)
    mse = mean_squared_error(y_test, preds)
    print("Test set R^2 Score: ", score)
    print("MSE: ", mse)
    print("RMSE: ", mse**(0.5))
    if save_threshold is not None and score >= save_threshold:
        xgbr_s.save_model(name+'.json')
        input_frame.to_csv(name+'.csv')
        print("SAVED!")
        


In [74]:
def merge_with_given(df):
    given = pd.read_csv('datasets/IC_CLUSTERS_AND_MCA.csv')[['unitid']]
    df = pd.merge(given, df, how='inner', on='unitid')
    return df

In [77]:
fnce_data_final_features

,unitid,f2b02,f2c05,f2c06,f2d11,f2d12,f2e011,f2e021,f2e031,f2e041,f2e051,f2e061,f2e071,f2e081,expense_ps
0,100690,1.0,0.000000,0.099897,0.000000,0.000000,0.325631,0.0,0.000000,0.062455,0.118260,0.493376,0.000000,0.000000,0.002731
1,100937,1.0,0.120239,0.349113,0.000000,0.327833,0.308731,0.0,0.000000,0.070772,0.204256,0.197669,0.211686,0.006885,0.001540
2,101073,1.0,0.000000,0.066926,0.000000,0.178585,0.309677,0.0,0.000000,0.043623,0.084801,0.300184,0.261713,0.000000,0.001999
3,101189,1.0,0.012823,0.267299,0.000000,0.075618,0.394805,0.0,0.000000,0.071439,0.210219,0.199167,0.100217,0.000000,0.000673
4,101365,1.0,0.000000,0.186007,0.000000,0.000000,0.403997,0.0,0.000000,0.056023,0.298759,0.169485,0.000000,0.000000,0.002060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,495192,1.0,0.000000,0.135527,0.000331,0.000000,0.593088,0.0,0.000000,0.022920,0.216563,0.167430,0.000000,0.000000,0.002188
1649,495271,1.0,0.000000,0.000000,0.000000,0.000000,0.078544,0.0,0.000000,0.202271,0.139390,0.497373,0.000000,0.000000,0.037077
1650,495280,1.0,0.038882,0.011565,0.027386,0.065732,0.391690,0.0,0.000000,0.097062,0.087589,0.343262,0.080397,0.000000,0.070018
1651,495314,1.0,0.000000,0.000000,0.000000,0.000000,0.228719,0.0,0.010244,0.345769,0.109479,0.305789,0.000000,0.000000,0.004717


In [85]:
def test_set(datapath, target, name, second = None, log = False, div_by_student = True, div_by_expense = True):
    df = pd.read_csv(datapath, low_memory=False)
    print(target)
    df_subset = df[['unitid', target]]
    df_subset.dropna(inplace=True, axis = 0)
    print(df_subset.shape)
    df_subset = df_subset.groupby('unitid').mean().reset_index()
    if second != None:
        df_subset[target] /= df[second]
        df_subset = df_subset.drop([second], axis=1)
    if div_by_student:
        df_subset = pd.merge(df_subset, total_students, how='inner', on='unitid')
        df_subset[target] /= df_subset['efytotlt']
        df_subset = df_subset.drop(['efytotlt'], axis = 1)
    if div_by_expense:
        expenses = fnce_data_final[['f2e131', 'unitid']]
        df_subset = pd.merge(df_subset, expenses, on='unitid', how='inner')
        print(df_subset.columns)
        df_subset[target] /= df_subset['f2e131']
        print(df_subset.shape)
        df_subset = df_subset.drop(['f2e131'], axis = 1)
    if log:
        df_subset[target] = np.log(df_subset[target])
        df_subset.replace([np.inf, -np.inf], np.nan, inplace=True)
        print(get_missing(df_subset))
        # df_subset = df_subset.fillna(df_subset.median())
        df_subset.dropna(inplace=True)
    merged = pd.merge(df_subset, fnce_data_final_features, how='inner', on='unitid')
    print(merged.shape)
    print(get_missing(merged))
    merged.dropna(inplace=True)
    print(merged)
    # merged = merge_with_given(merged)
    X = merged.drop(['unitid', target], axis = 1)
    y = merged[target]
    # scaler = StandardScaler()
    # X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

    # for c in X.columns:
    #     merged[c] = X[c]
    # y[np.array(np.where(y ==0))] = 1e-9
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    xgb_test(X, y, merge_with_given(merged), name, 0.74)
    # test(X, y)


In [117]:
test_set("datasets/Graduation Rates/GR_2015-2021_data.csv ", 'grtotlt', 'grad_rate_per_dollar', div_by_student=True, div_by_expense=True, log = True)

grtotlt
(369741, 2)
Index(['unitid', 'grtotlt', 'f2e131'], dtype='object')
(1395, 3)
         Percent Of Missing Values
unitid                         0.0
grtotlt                        0.0
(1334, 15)
            Percent Of Missing Values
unitid                            0.0
grtotlt                           0.0
f2c05                             0.0
f2c06                             0.0
f2d11                             0.0
f2d12                             0.0
f2e011                            0.0
f2e021                            0.0
f2e031                            0.0
f2e041                            0.0
f2e051                            0.0
f2e061                            0.0
f2e071                            0.0
f2e081                            0.0
expense_ps                        0.0
      unitid    grtotlt     f2c05     f2c06     f2d11     f2d12    f2e011  \
0     100690 -20.505651  0.000000  0.099897  0.000000  0.000000  0.325631   
1     100937 -19.034011  0.120239  0.

C:\Users\dragonblade62\AppData\Local\Temp\ipykernel_14188\252550041.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.dropna(inplace=True, axis = 0)


{'n_estimators': 30, 'max_depth': 10, 'grow_policy': 'depthwise', 'eta': 0.25}
Test set R^2 Score:  0.7495204742636008
MSE:  0.7001861267395881
RMSE:  0.8367712511430995
SAVED!
